In [2]:
!pip install -q transformers

# Training Large Language Models

## Building a Tiny Language Model

In [3]:
from collections import defaultdict

# Function to compute bigram probabilities from a given corpus
def compute_bigram_probabilities(corpus):
    # Create a dictionary to store the bigram counts
    bigram_counts = defaultdict(lambda: defaultdict(int))
    # defaultdict creates missing items instead of throwing KeyError like dict

    # Iterate over each sentence in the corpus
    for sentence in corpus:
        words = sentence.split()  # Tokenize the sentence into words

        # Iterate over each word pair and update the bigram counts
        for i in range(len(words) - 1):
            current_word = words[i]
            next_word = words[i + 1]
            bigram_counts[current_word][next_word] += 1

    # Create a dictionary to store the bigram probabilities
    bigram_probabilities = defaultdict(lambda: defaultdict(float))

    # Iterate over each word and its following word in the bigram counts
    for current_word, next_words in bigram_counts.items():
        total_count = sum(next_words.values())
        for next_word, count in next_words.items():
            bigram_probabilities[current_word][next_word] = count / total_count

    return bigram_probabilities

# Input corpus
corpus = ["Peter is happy",
          "Anna is happy",
          "Anna is sad",
          "Anna is good"]

# Compute bigram probabilities
bigram_probabilities = compute_bigram_probabilities(corpus)

# Create dictionaries to store the highest and lowest probabilities for each word
highest_probabilities = {}
lowest_probabilities = {}

# Iterate over each word and its following word in the bigram probabilities
for current_word, next_words in bigram_probabilities.items():
    # Find the word with the highest probability for the current word
    highest_probability = max(next_words, key=next_words.get)
    highest_probabilities[current_word] = highest_probability

    # Find the word with the lowest probability for the current word
    lowest_probability = min(next_words, key=next_words.get)
    lowest_probabilities[current_word] = lowest_probability

# Generate a 2-word sentence beginning with the prompt
prompt = "Peter"
hword = prompt  # for high-probability sentence
lword = prompt  # for low-probability sentence
hsentence = prompt + " "
lsentence = prompt + " "

# Generate the highest-probability and lowest-probability sentences
for _ in range(2):
    hword = highest_probabilities[hword]
    hsentence += hword + " "
    lword = lowest_probabilities[lword]
    lsentence += lword + " "

# Print the generated sentences
print("Highest-probability sentence:", hsentence)
print("Lowest-probability sentence:", lsentence)

Highest-probability sentence: Peter is happy 
Lowest-probability sentence: Peter is sad 


## Building a Character-level Text Generation Model

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from transformers import TFAutoModelForCausalLM, AutoTokenizer


# Set a seed for reproducibility
tf.random.set_seed(42)

# Sample input text to train the model (you can use your own text)
input_text = """
Data science is an interdisciplinary field that uses scientific methods, processes, algorithms, and systems to extract knowledge and insights from structured and unstructured data.
"""

In [ ]:
# Preprocess the text data
chars = sorted(set(input_text))
char_to_idx = {char: idx for idx, char in enumerate(chars)}
idx_to_char = {idx: char for char, idx in char_to_idx.items()}
num_chars = len(chars)
input_seq_length = 100
step = 1

input_sequences = []
output_chars = []

for i in range(0, len(input_text) - input_seq_length, step):
    input_seq = input_text[i: i + input_seq_length]
    output_seq = input_text[i + input_seq_length]
    input_sequences.append([char_to_idx[char] for char in input_seq])
    output_chars.append(char_to_idx[output_seq])

X = np.array(input_sequences)
y = tf.keras.utils.to_categorical(output_chars, num_classes=num_chars)

# One-hot encode the input data
X_one_hot = np.zeros((X.shape[0], input_seq_length, num_chars))
for i, seq in enumerate(X):
    for j, char_idx in enumerate(seq):
        X_one_hot[i, j, char_idx] = 1

# Build the LSTM text generator model
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(128, input_shape=(input_seq_length, num_chars)),
    tf.keras.layers.Dense(num_chars, activation='softmax')
])

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam')

# Training the model
epochs = 100
batch_size = 64

model.fit(X_one_hot, y, epochs=epochs, batch_size=batch_size)


Epoch 1/100
2/2 [==============================] - 9s 34ms/step - loss: 3.2549
Epoch 2/100
2/2 [==============================] - 0s 23ms/step - loss: 3.2332
Epoch 3/100
2/2 [==============================] - 0s 34ms/step - loss: 3.2106
Epoch 4/100
2/2 [==============================] - 0s 26ms/step - loss: 3.1871
Epoch 5/100
2/2 [==============================] - 0s 25ms/step - loss: 3.1505
Epoch 6/100
2/2 [==============================] - 0s 31ms/step - loss: 3.0738
Epoch 7/100
2/2 [==============================] - 0s 30ms/step - loss: 2.9523
Epoch 8/100
2/2 [==============================] - 0s 28ms/step - loss: 2.9123
Epoch 9/100
2/2 [==============================] - 0s 21ms/step - loss: 2.8423
Epoch 10/100
2/2 [==============================] - 0s 36ms/step - loss: 2.8253
Epoch 11/100
2/2 [==============================] - 0s 38ms/step - loss: 2.8211
Epoch 12/100
2/2 [==============================] - 0s 24ms/step - loss: 2.8217
Epoch 13/100
2/2 [==============================]

In [ ]:
# Generate text using the trained model
def generate_text(model, seed_text, num_chars_to_generate=100):
    generated_text = seed_text
    for _ in range(num_chars_to_generate):
        x_pred = np.zeros((1, input_seq_length, num_chars))
        for t, char in enumerate(generated_text[-input_seq_length:]):
            x_pred[0, t, char_to_idx[char]] = 1.0

        preds = model.predict(x_pred, verbose=0)[0]
        next_char_idx = np.argmax(preds)
        next_char = idx_to_char[next_char_idx]
        generated_text += next_char
    return generated_text

In [ ]:
# Test the text generation
seed_text = "Data science is"
generated_text = generate_text(model, seed_text, num_chars_to_generate=200)
print(generated_text)

Data science issss  s                                         sss     sssssss                     t tt ttt  a aatttttt tttttttttttaadaaaaa






sttteettrtrea nnnn e   nnssts m trtructudd  annnsttruutred aaaa




ss


## Improving Model with Word Tokenization

In [ ]:
# Preprocess the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts([input_text])

# Vocabulary size (number of unique words in the input text)
num_words = len(tokenizer.word_index) + 1

# Convert text to sequences of word indices
input_sequences = tokenizer.texts_to_sequences([input_text])[0]
input_seq_length = 10  # The number of words in each input sequence
step = 1

# Prepare input sequences and output words
X = []
y = []
for i in range(0, len(input_sequences) - input_seq_length, step):
    input_seq = input_sequences[i : i + input_seq_length]
    output_seq = input_sequences[i + input_seq_length]
    X.append(input_seq)
    y.append(output_seq)

X = np.array(X)
y = np.array(y)

# Build the LSTM text generator model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=num_words, output_dim=100, input_length=input_seq_length),
    tf.keras.layers.LSTM(128),
    tf.keras.layers.Dense(num_words, activation='softmax')
])

# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')

# Training the model
epochs = 100
batch_size = 64

model.fit(X, y, epochs=epochs, batch_size=batch_size)



Epoch 1/100
1/1 [==============================] - 4s 4s/step - loss: 3.0857
Epoch 2/100
1/1 [==============================] - 0s 26ms/step - loss: 3.0748
Epoch 3/100
1/1 [==============================] - 0s 15ms/step - loss: 3.0636
Epoch 4/100
1/1 [==============================] - 0s 18ms/step - loss: 3.0517
Epoch 5/100
1/1 [==============================] - 0s 16ms/step - loss: 3.0389
Epoch 6/100
1/1 [==============================] - 0s 21ms/step - loss: 3.0245
Epoch 7/100
1/1 [==============================] - 0s 16ms/step - loss: 3.0082
Epoch 8/100
1/1 [==============================] - 0s 15ms/step - loss: 2.9893
Epoch 9/100
1/1 [==============================] - 0s 16ms/step - loss: 2.9671
Epoch 10/100
1/1 [==============================] - 0s 18ms/step - loss: 2.9406
Epoch 11/100
1/1 [==============================] - 0s 16ms/step - loss: 2.9086
Epoch 12/100
1/1 [==============================] - 0s 16ms/step - loss: 2.8697
Epoch 13/100
1/1 [==============================] -

In [ ]:
# Generate text using the trained model
def generate_text(model, seed_text, num_words_to_generate=50):
    generated_text = seed_text
    for _ in range(num_words_to_generate):
        input_seq = tokenizer.texts_to_sequences([generated_text])[0]
        input_seq = pad_sequences([input_seq], maxlen=input_seq_length, padding='pre')

        preds = model.predict(input_seq)[0]
        next_word_idx = np.argmax(preds)
        next_word = tokenizer.index_word[next_word_idx]
        generated_text += " " + next_word
    return generated_text

# Test the text generation
seed_text = "Data science is"
generated_text = generate_text(model, seed_text, num_words_to_generate=50)
print(generated_text)

1/1 [==============================] - 0s 38ms/step
Data science is and and and and and systems systems to and and from and and and unstructured data data and and and and and and systems processes to and from from from structured and unstructured unstructured unstructured data and processes processes algorithms and systems processes to extract insights from structured structured and


Take 3: Try on larger dataset

In [ ]:
import requests

response = requests.get('https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt')
shakespeare_data = response.text

In [ ]:
# Sample input text to train the model (you can use your own text)
input_text = shakespeare_data

# Preprocess the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts([input_text])

# Vocabulary size (number of unique words in the input text)
num_words = len(tokenizer.word_index) + 1

# Convert text to sequences of word indices
input_sequences = tokenizer.texts_to_sequences([input_text])[0]
input_seq_length = 10  # The number of words in each input sequence
step = 1

# Prepare input sequences and output words
X = []
y = []
for i in range(0, len(input_sequences) - input_seq_length, step):
    input_seq = input_sequences[i : i + input_seq_length]
    output_seq = input_sequences[i + input_seq_length]
    X.append(input_seq)
    y.append(output_seq)

X = np.array(X)
y = np.array(y)

# Build the LSTM text generator model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=num_words, output_dim=100, input_length=input_seq_length),
    tf.keras.layers.LSTM(128),
    tf.keras.layers.Dense(num_words, activation='softmax')
])

# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')

# Training the model
epochs = 10
batch_size = 4096

model.fit(X, y, epochs=epochs, batch_size=batch_size)



Epoch 1/10
50/50 [==============================] - 15s 237ms/step - loss: 8.0731
Epoch 2/10
50/50 [==============================] - 11s 227ms/step - loss: 6.8210
Epoch 3/10
50/50 [==============================] - 9s 176ms/step - loss: 6.8009
Epoch 4/10
50/50 [==============================] - 6s 127ms/step - loss: 6.7963
Epoch 5/10
50/50 [==============================] - 8s 168ms/step - loss: 6.7917
Epoch 6/10
50/50 [==============================] - 5s 111ms/step - loss: 6.7789
Epoch 7/10
50/50 [==============================] - 6s 109ms/step - loss: 6.7284
Epoch 8/10
50/50 [==============================] - 4s 88ms/step - loss: 6.6705
Epoch 9/10
50/50 [==============================] - 4s 74ms/step - loss: 6.6280
Epoch 10/10
50/50 [==============================] - 5s 94ms/step - loss: 6.5891


In [ ]:
# Generate text using the trained model
def generate_text(model, seed_text, num_words_to_generate=50):
    generated_text = seed_text
    for _ in range(num_words_to_generate):
        input_seq = tokenizer.texts_to_sequences([generated_text])[0]
        input_seq = pad_sequences([input_seq], maxlen=input_seq_length, padding='pre')

        preds = model.predict(input_seq)[0]
        next_word_idx = np.argmax(preds)
        next_word = tokenizer.index_word[next_word_idx]
        generated_text += " " + next_word
    return generated_text

# Test the text generation
seed_text = "He had rather see the swords"
generated_text = generate_text(model, seed_text, num_words_to_generate=20)
print(generated_text)

1/1 [==============================] - 0s 38ms/step
He had rather see the swords and the lord the lord the lord the lord the lord the lord the lord the lord the lord the
